# Text Generation

Experimenting with a Text Generation Pipeline

In [1]:
from transformers import pipeline

In [2]:
# Use a pre-quantized model from the Hub
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-3B",
    device_map="auto",
    do_sample=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [4]:
def get_faker_mapping(properties):
    # Create a structured prompt
    prompt = """
You are a data helper.

Function Names:
name
first_name
last_name
email
phone
address
city
state
zip_code
country
company
job_title
credit_card_number
date_of_birth
text
random_number

EXAMPLE 1:
Properties:
fullName: Full name of person
businessEmail: Business email address
phoneNum: US phone number

Response:
fullName: name
businessEmail: email
phoneNum: phone

EXAMPLE 2:
Properties:
name: name of person
ccn: Credit card number
personalPhone: US phone number

Response:
name: name
ccn: credit_card_number
personalPhone: phone

EXAMPLE 3:
Properties:
firstName: first name of person
city: City of residence
personalPhone: US phone number

Response:
firstName: first_name
city: city
personalPhone: phone

Complete EXAMPLE 4:
Properties:
"""
    # Add properties to prompt
    for prop, desc in properties:
        prompt += f"{prop}: {desc}\n"
    
    prompt += "\nResponse:"
    
    # Generate response
    response = pipe(
        prompt,
        max_new_tokens=50
    )
    
    result = response[0]['generated_text']
    
    # Logic to convert to a dict
    response_text = result.split("Complete EXAMPLE 4:")[1].split("Complete EXAMPLE 5:")[0].split("Response:")[1].strip()
    
    return {line.split(":", 1)[0].strip(): line.split(":", 1)[1].strip() for line in response_text.splitlines() if ":" in line}
    
# Example use
properties = [
    ("name", "users first name"),
    ("email", "Business email address"),
    ("phone", "US phone number")
]

result = get_faker_mapping(properties)

for key, desc in properties:
    print(result.get(key))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


first_name
email
phone
